# Making a GAN to generate MNIST digits

## Boring imports and datasets

In [1]:
import torch 
import torch.nn as nn 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import sklearn.model_selection as ms
from tqdm import tqdm

In [2]:
class MNIST_Dataset(Dataset):
    def __init__(self, metadata_df, images):
        self.metadata_df = metadata_df
        self.images = images / 255

    def __len__(self):
        return len(self.metadata_df)

    def __getitem__(self,idx):
        idx = int(idx)

        image = torch.tensor(images[idx])
        # label = torch.tensor(self.metadata_df.loc[idx, 'labels'])
        label = torch.tensor([1])

        return image, label

In [3]:
metadata = pd.read_csv('data/MNIST/metadata.csv')
train_metadata, test_metadata = ms.train_test_split(metadata, test_size=0.2, train_size=0.8, random_state=19, shuffle=True, stratify=metadata['labels'])

images = np.load('data/MNIST/images.npy')

In [4]:
train_dataset = MNIST_Dataset(train_metadata.reset_index(), images[train_metadata.index])
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=256)

## Fun part: models

In [5]:
# I have no idea if this is the way to make the generator/discriminator but I'll do it in a encoder/decoder way since thats the intuition I have about it
class Generator(nn.Module):
    def __init__(self,num_latent_features):
        super(Generator, self).__init__()

        self.first_linear = nn.Sequential(
            nn.Linear(num_latent_features,1024),
            nn.ReLU() ) 

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            
            nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        
            nn.ConvTranspose2d(64, 1, kernel_size=2, stride=2),
            nn.ReLU(),
        
            nn.Conv2d(1, 1, kernel_size=5, stride=1, padding=0),
        )

    def forward(self, x):
        x = self.first_linear(x).view(-1,1024,1,1)
        x = self.decoder(x);
        return x

class Discriminator(nn.Module):
    def __init__(self,num_embeddings):
        super(Discriminator, self).__init__()
        self.num_embeddings = num_embeddings

        self.encoder = nn.Sequential(
            # 32x32 to 28x28
            nn.Conv2d(1, 64, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            
            # 28x28 to 14x14
            nn.Conv2d(64, 128, kernel_size=2, stride=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
        
            # 14x14 to 7x7
            nn.Conv2d(128, 256, kernel_size=2, stride=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
        
            # 7x7 to 3x3
            nn.Conv2d(256, 512, kernel_size=2, stride=2),
            nn.BatchNorm2d(512),
            nn.ReLU(),
        
            # 3x3 to 1x1
            nn.Conv2d(512, self.num_embeddings, kernel_size=3, stride=1, padding=0),
            nn.ReLU()
        )

        self.classifier = nn.Sequential(
            nn.Linear(self.num_embeddings, 512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128,1),
        )

    def forward(self, x):
        x = self.encoder(x).view(-1,self.num_embeddings)
        x = self.classifier(x)
        return x

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

g = Generator(num_latent_features = 10).to(device)
d = Discriminator(num_embeddings = 1024).to(device)

criterion = nn.BCEWithLogitsLoss()

d_optimizer = nn.optim.Adam(d.parameters(), lr=1e-5, weight_decay=0)
g_optimizer = nn.optim.Adam(g.parameters(), lr=1e-5, weight_decay=0)

AttributeError: module 'torch.nn' has no attribute 'Adam'

In [15]:
dups = g(torch.randn(10,10)) # (10,1,28,28)

## discriminator functions

In [17]:
def d_train(batch,generator, discriminator, device,optimizer, criterion):
    discriminator.train()
    generator.eval()
    
    images, labels = batch
    images, labels = images.to(device), labels.to(device)

    random_latent_spaces = torch.distributions.Normal(0, 1).sample((len(labels), 10))
    with torch.no_grad():
        fake_images = generator(random_latent_spaces)

    images, labels = torch.cat([images, fake_images],0), torch.cat([labels, torch.zeros((len(labels), ))])

    optimizer.zero_grad()
    
    outputs = discriminator(images)
    loss = criterion(outputs, labels)
    optimizer.backward(loss)
    optimizer.step()
    
    with torch.no_grad():
        metric = ((torch.sigmoid(outputs) > 0.5) == labels) / len(labels)

    return loss,metric

## generator functions

In [18]:
def g_train(batch_size, generator ,discriminator, device,optimizer, criterion):
    generator.train()
    discriminator.eval()

    optimizer.zero_grad()

    random_latent_spaces = torch.distributions.Normal(0, 1).sample((batch_size, 10))
    outputs = discriminator(random_latent_spaces)

    with torch.no_grad():
        crediblity = discriminator(outputs)


        
    loss = criterion(crediblity, torch.ones((batch_size,)))
    optimizer.backward(loss)
    optimizer.step()

    with torch.no_grad():
        random_idxs = torch.randint(0,batch_size,(5,))
        for i,idx in enumerate(random_idxs):
            plt.subplot(1,5,i)
            plt.imshow(outputs[idx])
        plt.show()
            

    return loss,metric

In [10]:
for i, batch in enumerate(tqdm(train_loader, total = len(train_loader))):
    d_train(batch,g, d, device ,d_optimizer, criterion)
    g_train(32, g, d, device ,d_optimizer, criterion)

  9%|▉         | 20/219 [00:00<00:02, 97.48it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


 23%|██▎       | 51/219 [00:00<00:01, 132.39it/s]

21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51


 38%|███▊      | 84/219 [00:00<00:00, 148.18it/s]

...

151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183


 99%|█████████▊| 216/219 [00:01<00:00, 158.87it/s]

184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216


100%|██████████| 219/219 [00:01<00:00, 148.53it/s]

217
218
